In [1]:
## imports
import glob, os

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

import requests
from io import BytesIO
from PIL import Image
import numpy as np
pylab.rcParams['figure.figsize'] = 20, 12
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.engine.predictor_glip import GLIPDemo

e:\Git\ML\VideoAnalytics\.pixi\envs\default\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\Git\ML\VideoAnalytics\GLIP\maskrcnn_benchmark\layers\deform_conv.py:313: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
e:\Git\ML\VideoAnalytics\GLIP\maskrcnn_benchmark\layers\deform_conv.py:378: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
e:\Git\ML\VideoAnalytics\GLIP\maskrcnn_benchmark\layers\deform_conv.py:428: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @cus

In [2]:
def load(file):
    pil_image = Image.open(file).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def load_url(url):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    response = requests.get(url)
    pil_image = Image.open(BytesIO(response.content)).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def imshow(img, caption):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")
    plt.figtext(0.5, 0.09, caption, wrap=True, horizontalalignment='center', fontsize=20)

def imsave(img, path):
    plt.imsave(path, img[:, :, [2, 1, 0]])

In [7]:
config_file = "configs/pretrain/glip_Swin_L.yaml"
weight_file = "../models/glip_large_model.pth"

# update the config options with the config file
# manual override some options
cfg.local_rank = 0
cfg.num_gpus = 1
cfg.merge_from_file(config_file)
cfg.merge_from_list(["MODEL.WEIGHT", weight_file])
cfg.merge_from_list(["MODEL.DEVICE", "cuda"])

glip_demo = GLIPDemo(
    cfg,
    min_image_size=800,
    confidence_threshold=0.7,
    show_mask_heatmaps=False
)

VISION BACKBONE USE GRADIENT CHECKPOINTING:  False
LANGUAGE BACKBONE USE GRADIENT CHECKPOINTING:  False
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B


e:\Git\ML\VideoAnalytics\GLIP\maskrcnn_benchmark\utils\checkpoint.py:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cuda"

In [8]:
#caption = 'person . bicycle . car . motorcycle . airplane . bus . train . truck . boat . traffic light . fire hydrant . stop sign . parking meter . bench . bird . cat . dog . horse . sheep . cow . elephant . bear . zebra . giraffe . backpack . umbrella . handbag . tie . suitcase . frisbee . skis . snowboard . sports ball . kite . baseball bat . baseball glove . skateboard . surfboard . tennis racket . bottle . wine glass . cup . fork . knife . spoon . bowl . banana . apple . sandwich . orange . broccoli . carrot . hot dog . pizza . donut . cake . chair . couch . potted plant . bed . dining table . toilet . tv . laptop . mouse . remote . keyboard . cell phone . microwave . oven . toaster . sink . refrigerator . book . clock . vase . scissors . teddy bear . hair drier . toothbrush .'
#caption = ['man in back wearing grey shirt . women walking in background . ', 'man in yellow shirt . blue shirt . ']
caption = ['small ID card with photo']

In [9]:
#for files in [glob.glob(e) for e in ['INPUT/*.jpg', 'INPUT/*.png', 'INPUT/*.JPG', 'INPUT/*.PNG']]:
# for files in [glob.glob(e) for e in ['INPUT/*.png']]:
files = ['../data/input_image.png']
for i, file in enumerate(files):
    image = load(file)
    result, _ = glip_demo.run_on_web_image(image, caption[i], 0.5)
    #imshow(result, caption)
    imsave(result, "RESULTS/"+file.split("/")[-1])

[[[0, 13]], [[19, 24]]]


e:\Git\ML\VideoAnalytics\.pixi\envs\default\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
e:\Git\ML\VideoAnalytics\.pixi\envs\default\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
e:\Git\ML\VideoAnalytics\.pixi\envs\default\Lib\site-packages\transformers\modeling_utils.py:1161: FutureWarning: The `devi

RuntimeError: Not compiled with GPU support
Exception raised from modulated_deform_conv_forward at E:\Git\ML\VideoAnalytics\GLIP\maskrcnn_benchmark\csrc\deform_conv.h:145 (most recent call first):
00007FFEAA07462900007FFEAA074580 c10.dll!c10::Error::Error [<unknown file> @ <unknown line number>]
00007FFEAA07416A00007FFEAA074110 c10.dll!c10::detail::torchCheckFail [<unknown file> @ <unknown line number>]
00007FFDB3D7ABFA00007FFDB3D65C60 _C.cp312-win_amd64.pyd!c10::ivalue::Object::operator= [<unknown file> @ <unknown line number>]
00007FFDB3D85C5100007FFDB3D7AE20 _C.cp312-win_amd64.pyd!PyInit__C [<unknown file> @ <unknown line number>]
00007FFDB3D8238000007FFDB3D7AE20 _C.cp312-win_amd64.pyd!PyInit__C [<unknown file> @ <unknown line number>]
00007FFDB3D8241400007FFDB3D7AE20 _C.cp312-win_amd64.pyd!PyInit__C [<unknown file> @ <unknown line number>]
00007FFDB3D77E8D00007FFDB3D65C60 _C.cp312-win_amd64.pyd!c10::ivalue::Object::operator= [<unknown file> @ <unknown line number>]
00007FFEACDC28E3